In [1]:
import pandas as pd
import os
import dask.dataframe as dd
import warnings
from tqdm import tqdm

In [5]:
warnings.filterwarnings('ignore')

In [3]:
#TEXT_EMBEDDING_PATH = "../data/artifacts/google_bert_base_multilingual_cased/google_bert_base_multilingual_cased/bert_base_multilingual_cased.parquet"

In [6]:
SOURCE = "ebnerd_large"

In [22]:
behave_dfs = []
for split in ["train", "validation"]:
    DATA_PATH = f"../data/{SOURCE}/{split}/"
    _df_behave = pd.read_parquet(f"{DATA_PATH}/behaviors.parquet")
    _df_behave = _df_behave[["user_id", "impression_time", "article_ids_clicked", "next_read_time"]]
    # as the most clicked article is 1, we can take only the first article
    _df_behave["article_id"] = _df_behave["article_ids_clicked"].apply(lambda x: x[0])
    _df_behave = _df_behave.drop(columns=["article_ids_clicked"])
    _df_behave.columns = ["user_id", "impression_ts", "article_read_time", "article_id"]
    behave_dfs.append(_df_behave)
behave_dfs = pd.concat(behave_dfs, ignore_index=True)

In [23]:
behave_dfs.to_parquet(f"{DATA_PATH}/all.parquet")

In [26]:
# expanding user's history
split = "validation"
#DATA_PATH = f"../data/{SOURCE}/{split}/"
DATA_PATH = f"../data/ebnerd_testset/ebnerd_testset/test"
df_history = pd.read_parquet(f"{DATA_PATH}/history.parquet")
#df_history = pd.read_parquet(f"{DATA_PATH}/history_exploded.parquet") 

In [29]:
df_history["article_id_fixed"].apply(lambda x: len(x)).sum()

116825984

In [21]:
ddf = dd.from_pandas(df_history, npartitions=1000)

In [22]:
# Perform the explode operation
ddf_exploded = ddf.explode(['impression_time_fixed', 'scroll_percentage_fixed', 'article_id_fixed', 'read_time_fixed'])

In [23]:
# Convert back to pandas DataFrame if needed
df_history = ddf_exploded.compute()

In [24]:
df_history.to_parquet(f"{DATA_PATH}/history_exploded.parquet")

In [25]:
df_history

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,40107,2023-05-11 07:51:01,15.0,9676294,25.0
0,40107,2023-05-13 20:09:06,100.0,9763942,39.0
0,40107,2023-05-13 20:09:46,100.0,9763968,148.0
0,40107,2023-05-13 20:12:14,NaN,9763905,0.0
0,40107,2023-05-14 05:22:14,12.0,9764070,11.0
...,...,...,...,...,...
807676,2081381,2023-05-18 10:20:38,NaN,9771242,1.0
807676,2081381,2023-05-18 10:20:42,100.0,9771253,46.0
807676,2081381,2023-05-18 10:21:31,100.0,9771065,64.0
807676,2081381,2023-05-18 10:22:38,94.0,9770792,71.0


In [17]:
#df_history = df_history.explode(['impression_time_fixed', 'scroll_percentage_fixed', 'article_id_fixed', 'read_time_fixed'], ignore_index=True)
df_history = df_history[["user_id", "impression_time_fixed", "article_id_fixed", "read_time_fixed"]]
df_history.columns = ["user_id", "impression_ts", "article_id", "article_read_time"]
df_history

,user_id,impression_ts,article_id,article_read_time
0,10029,2023-04-28 06:16:57,9735579,28.0
0,10029,2023-04-28 06:17:31,9739888,24.0
0,10029,2023-04-28 06:18:12,9739471,11.0
0,10029,2023-04-28 06:18:36,9739864,107.0
0,10029,2023-04-28 06:20:49,9738441,8.0
...,...,...,...,...
788089,1541617,2023-05-08 07:31:13,9753603,18.0
788089,1541617,2023-05-08 07:33:23,9753684,117.0
788089,1541617,2023-05-08 07:35:27,9753668,115.0
788089,1541617,2023-05-08 07:37:32,9750772,11.0


In [24]:
# merging history with the main dataframe
df_behave = pd.concat([df_behave, df_history], ignore_index=True)
df_behave = df_behave.sort_values(by=["user_id", "impression_ts"])
df_behave

,user_id,impression_ts,article_read_time,article_id
131668681,10000,2023-05-07 21:04:52,16.0,9752998
131668682,10000,2023-05-10 14:08:01,18.0,9755010
131668683,10000,2023-05-11 13:29:45,24.0,9759219
131668684,10000,2023-05-11 17:35:14,126.0,9760137
131668685,10000,2023-05-11 17:37:21,81.0,9759355
...,...,...,...,...
8081675,2590692,2023-05-23 21:20:48,23.0,9220931
8081673,2590692,2023-05-23 21:23:16,28.0,9778310
8081676,2590692,2023-05-23 21:25:25,64.0,9759955
8081677,2590692,2023-05-23 21:26:21,13.0,9522337


In [4]:
#df_behave.to_parquet(f"{DATA_PATH}/all_hist.parquet")
df_behave = pd.read_parquet(f"../data/{SOURCE}/all_hist.parquet")

In [5]:
articles = pd.read_parquet(f"../data/{SOURCE}/articles.parquet")

In [6]:
# fill total_read_time with median, where it is missing
articles = articles.fillna(articles["total_read_time"].median())
articles["article_ctr"] = articles["total_pageviews"] / articles["total_inviews"]
# fill ctr with median, where it is missing
articles = articles.fillna(articles["article_ctr"].median())

In [7]:
articles = articles[["article_id", "premium", "article_type", "topics", "category_str", "total_read_time", "sentiment_score", "article_ctr"]]

In [8]:
articles.columns = ["article_id", "article_is_premium", "article_type", "article_topics", "article_category", "article_total_read_time", "article_sentiment", "article_ctr"]

In [9]:
# merging articles with the main dataframe
df_behave = df_behave.merge(articles, on="article_id", how="left")

In [13]:
df_behave

,user_id,impression_ts,article_read_time,article_id,article_is_premium,article_type,article_topics,article_category,article_total_read_time,article_sentiment,article_ctr
0,10000,2023-05-07 21:04:52,16.0,9752998,False,article_default,"[Privat virksomhed, Transportmiddel, Bil, Erhv...",nyheder,14592461.0,0.7704,0.232832
1,10000,2023-05-10 14:08:01,18.0,9755010,False,article_default,"[Kriminalitet, Underholdning, Film og tv, Pers...",underholdning,4219415.0,0.8539,0.194664
2,10000,2023-05-11 13:29:45,24.0,9759219,False,article_default,"[Erhverv, Økonomi, Mikro, Makro]",nyheder,16611023.0,0.9762,0.227514
3,10000,2023-05-11 17:35:14,126.0,9760137,False,article_default,"[Kriminalitet, Transportmiddel, Bil, Politik, ...",nationen,4739068.0,0.9507,0.220218
4,10000,2023-05-11 17:37:21,81.0,9759355,False,article_default,"[Kriminalitet, Personfarlig kriminalitet]",krimi,37657040.0,0.9703,0.324828
...,...,...,...,...,...,...,...,...,...,...,...
137246827,2590692,2023-05-23 21:20:48,23.0,9220931,True,article_default,"[Livsstil, Erotik]",sex_og_samliv,2334108.0,0.8240,1.000000
137246828,2590692,2023-05-23 21:23:16,28.0,9778310,False,article_default,"[Begivenhed, Sport, Sportsbegivenhed]",sport,1664025.0,0.4819,0.179211
137246829,2590692,2023-05-23 21:25:25,64.0,9759955,True,article_default,"[Kendt, Underholdning, Transportmiddel, Kultur...",underholdning,4437972.0,0.8218,0.062221
137246830,2590692,2023-05-23 21:26:21,13.0,9522337,True,article_default,"[Erhverv, Kendt, Økonomi, Mikro]",underholdning,2334108.0,0.6150,1.000000


In [ ]:
# loading text embeddings
#emb_df = pd.read_parquet(TEXT_EMBEDDING_PATH)
#emb_df.columns = ["article_id", "article_emb"]

In [ ]:
#emb_df

In [ ]:
# merging text embeddings with the main dataframe
#df_behave = df_behave.merge(emb_df, on="article_id", how="left")
#df_behave

In [ ]:
# Helper function to split the DataFrame into chunks of size 'n'
def chunker(df, chunk_size):
    return [df.iloc[i:i + chunk_size] for i in range(0, df.shape[0], chunk_size)]

# Assuming df_behave is your DataFrame
df_behave_sorted = df_behave.sort_values('impression_ts', ascending=False)

# Create a list to store the resulting chunks
chunks = []

# Iterate through each group, chunk them, and append to the list
for name, group in tqdm(df_behave_sorted.groupby('user_id')):
    user_chunks = chunker(group, 20)
    for i, chunk in enumerate(user_chunks):
        if len(chunk) >= 2:  # Ensure each session has at least two records
            chunk['session'] = f'{name}_session_{i+1}'  # Create unique session IDs
            chunks.append(chunk)

# Concatenate all chunks into a single DataFrame
df_result = pd.concat(chunks).reset_index(drop=True)

In [28]:
df_result = df_result.fillna(df_result["article_read_time"].median())

In [29]:
df_result.to_parquet(f"../data/{SOURCE}_modified/all.parquet", index=False)